In [ ]:
# Import library

# Packages required
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import fits
import os
from deconvbench import Deconvbench
from mpl_toolkits.axes_grid1 import make_axes_locatable
from amiral import instructment, utils, parameter, gradient, minimisation, array, plotting
from amiral.plotting import plotting_PSF_PSD as amiral_plt

# from plotting import plot_PSF_PSD as amiral_plt
from scipy.optimize import minimize 
from maoppy.utils import binning
import maoppy

%matplotlib inline

import tools

rcParams["figure.figsize"] = 20,33

In [ ]:
# Function for getting snr
def get_snr (array, noise):
    
    mean = np.mean(array)
    sig2 = np.std(noise)
    
    snr = mean / sig2
    
    return snr

def psd_object (param):
    
    rho = np.fft.fftshift(utils.dist(768))/param[1]
    psd_obj =  param[0]/ (np.power(rho,param[2]) + 1.)
    
    return psd_obj
    
def plot_psd_object(psd_obj): 
    
    fig, ax = plt.subplots()
    ycent = int((256*aosys_cls.samp_factor[0])//2)

    ax.plot(np.abs(psd_obj[ycent,...]))
    ax.set_title('PSD Object(total)')
    ax.axhline(y=1, color = 'r', ls = '--')
    
    pass

def create_psfao19_otf (otf_tel, guess, aosys_cls): 
    
    # Use PSFAO19 model to create a PSF
    psd_ao = aosys_cls.psd_residual_ao (guess = guess)
    psd_halo = aosys_cls.psd_residual_halo(r0 = guess[0])
    
    psd = psd_ao + psd_halo

    otf_atmo = aosys_cls.otf_atmo(psd)
    otf_total = otf_atmo*otf_tel
      
    return otf_atmo,otf_total

def zoom_array (array, size): 
    
    cent = np.shape(array)
    zoomed_array = array[cent[0]//2 - size:cent[0]//2 +size, cent[0]//2 - size:cent[0]//2 +size]
    
    return zoomed_array





In [ ]:
# PATH
wdir = "/Users/alau/Data/amiral_fits/VESTA/SNR/"
path_asteriod = '/Users/alau/IDLWorkspace/Data/Vesta_OASIS/2018-06-08T05_27_05.809.fits'


In [ ]:
# Parameters for the PSF
psf_dict = {
    "r0": 0.15,               
    "background": 0.,
    "amplitude": 2.0,
    "ax": 0.05,                            
    "beta": 1.5, 
    "mu": 0., 
    "rho0": 0., 
    "p": 0. 
}

# Variable
RON = 15. # CCD read-out noise standard-deviation [e-]
FLUX = 178112394513.1226 
DIMENSION = int(256)

flux_snr = 178112394513.1226 
print(flux_snr)

# psf_dict["r0"] = 0.1
# psf_dict["amplitude"] = 0.5

Get the object from the high resolution simulation

In [ ]:
data = fits.open(path_asteriod)
asteriod = data[0].data

Get some more ob

We need a PSF

In [ ]:
# Set up the telescope and produce a PSF
amiral_dict = {
    "r0": 0.2,                  
    "background": 0.,      
    "amplitude": 2.5,       
    "ax": 0.05,                            
    "beta": 1.5, 
    "mu": 0., 
    "rho0": 0., 
    "p": 3. 
}

aosys_dict = {
    'diameter': 8 , 
    'occ_ratio': 0.14 , 
    'no_acutuator' : 20, 
    'wavelength': 500, 
    'dimension': 256,
    'resolution_rad' : 2.083e-8
}


# aosys_dict = {
#     'diameter': 8 , 
#     'occ_ratio': 0.14 , 
#     'no_acutuator' : 34, 
#     'wavelength': 500, 
#     'dimension': 256,
#     'resolution_rad' : 1.1977272727272726e-07
# }

#'resolution_rad' : 1.25e-7 3.125e-8 1.1977272727272726e-07

# Passing parametpsd_arrayers from the telesope to the aosystem
aosys_cls = instructment.aoSystem( 
        diameter = aosys_dict['diameter'], occ_ratio = aosys_dict['occ_ratio'], 
        no_acutuator= aosys_dict['no_acutuator'], wavelength = aosys_dict['wavelength']*1e-9, 
        resolution_rad = aosys_dict['resolution_rad'], 
        dimension=aosys_dict['dimension'])  





psf_keys, psf_param = utils.dict2array(psf_dict)
amiral_keys, psf_guess = utils.dict2array(amiral_dict)

# Calibrating the flux
asteriod = array.scale_array(asteriod, 3)
asteriod_resize = asteriod/np.sum(asteriod)*flux_snr

Add Photon noise

In [ ]:
_test = aosys_cls.get_pix2freq(aosys_cls.samp_factor)


_xy = np.mgrid[0:256*3, 0:256*3].astype(float)
_xy[0] -= 256*3//2
_xy[1] -= 256*3//2

_fxy = _xy*_test

In [ ]:
rng = np.random.default_rng()
photon_noise = rng.poisson(asteriod_resize)
print("\nPhoton Noise of the object: ", np.sum((photon_noise)))

ft_photon_noise = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(photon_noise)))
ft_obj = np.fft.fft2(np.fft.ifftshift(asteriod_resize))

In [ ]:
print(psf_param)

psd_ao = aosys_cls.psd_residual_ao (psf_param)
psd_halo = aosys_cls.psd_residual_halo(psf_dict['r0'])

psd = psd_halo + psd_ao 

pupil = aosys_cls.get_pupil_plane()
otf_tel = aosys_cls.pupil_to_otf_tel(pupil)

integral, SR = aosys_cls.psd_integral(psd_ao, psf_dict['r0'])

otf_atmo = aosys_cls.otf_atmo(psd)

otf_total = np.fft.ifftshift(otf_atmo * otf_tel)

psf_total = np.real(np.fft.ifft2(otf_total))
print("Sum of the psf : %.2f" %(np.sum(psf_total)))

psf_tel = np.real(np.fft.ifft2(np.fft.ifftshift(otf_tel)))
print("Sum of the psf_tel :%.2f" %(np.sum(psf_tel)))

plt.imshow(np.real(np.log10(otf_total)))

In [ ]:
amiral_plt.plot_otf_slice(otf_atmo,otf_tel)

In [ ]:
psf_binned = aosys_cls.psfao(otf_total)
plt.imshow(np.fft.fftshift(np.log10(psf_binned)))
print(np.sum(psf_binned))

otf_binned = np.fft.fft2(psf_binned)

plt.imshow(np.real(np.fft.fftshift(np.log10(otf_binned))))

In [ ]:
psf_tel_binned = binning (psf_tel, aosys_cls.samp_factor[0])
plt.imshow(np.log10(psf_tel_binned))
print("Sum: %.2f" %(np.sum(psf_tel_binned)))

In [ ]:
plt.imshow(np.real(np.log10(ft_obj)))

Get the convoloved object for the additive noise model.

In [ ]:
# ft_conv_obj = np.fft.fftshift(ft_obj)*np.fft.fftshift(otf_binned)

# conv_obj = np.real(np.fft.ifft2(np.fft.ifftshift(ft_conv_obj)))


# rcParams['figure.figsize'] = 13 ,11

# fig, ax = plt.subplots(1,2)
# fig.tight_layout(pad=0.4, w_pad=0.6, h_pad=4.0)
# ax[0].imshow(np.real(np.log10(ft_conv_obj)))
# ax[1].imshow(np.real(np.log10(np.fft.fftshift(conv_obj))))

# print("Flux of the convoloved object: %f " %(np.sum(conv_obj)))

Calculate the noise and simulate realistic observation

In [ ]:
# rng = np.random.default_rng()
# photon_noise = rng.poisson(np.fft.fftshift(conv_obj))

# print("\nPhoton Noise of the object: ", np.sum((photon_noise)))

# gauss_noise = np.random.randn(DIMENSION,DIMENSION)*RON

# print("\nGaussian Noise of the object: ", np.sum((gauss_noise)))

# noise = gauss_noise + photon_noise

# fig, ax = plt.subplots(1,2)
# fig.tight_layout(pad=0.4, w_pad=0.6, h_pad=4.0)
# ax[0].imshow(photon_noise)
# ax[1].imshow(gauss_noise)

We are interested in SNR but I have no idea how to get it for now ...

In [ ]:
# # obs_image = noise + _obj
# obs_image = np.fft.fftshift(conv_obj) + noise

# # obs_image = conv_obj 

# empty_arr = np.zeros((DIMENSION,DIMENSION))+1.

# psd_noise = np.abs(np.fft.fftshift(np.fft.fft2(noise)))**2
# psd_asteroid = np.abs(np.fft.fftshift(np.fft.fft2(obs_image)))**2

# snr = psd_asteroid/psd_noise

# # est_obj = tools.wiener_filter(np.fft.fft2(obs_image),otf_total, snr)

# # fig, ax = plt.subplots(1,2)

# # rcParams['figure.figsize'] = 33 ,24

# # im = ax[0].imshow(np.log10(snr))
# # fig.colorbar(im, ax=ax[0])

# # im1 = ax[1].imshow(np.real(np.log10(est_obj)))
# # fig.colorbar(im1, ax=ax[1])

# plt.imshow(np.real(obs_image))

Get simulated object and see the noise too

In [ ]:
# img = obs_image

wdir = "/Users/alau/Data/amiral_fits/VESTA/SNR/2021apr23/case_9/"
output = "2018-06-08T05_27_05.809_199.fits"

# noise_output = "2018-06-08T05_27_05.809_noise_90.fits"

_data = fits.open(wdir+output)
data = _data[0].data

print(_data[0].header)

# _noise_data = fits.open(wdir+noise_output)
# _noise_data = _noise_data[0].data

# img = _data
img = data

obs_image = img
# noise = _noise_data

amiralparam = parameter.amiralParam(img ,guess = psf_guess, aosys = aosys_cls)

# k_hat = (1e6**2)
# psd_noise = np.sum(np.abs(noise)**2)

# mu = psd_noise / k_hat

plt.imshow(zoom_array(data, 150), cmap = 'gray')

In [ ]:
# fig, ax = plt.subplots(2,2)

# rcParams['figure.figsize'] = 33 ,24
# divider = make_axes_locatable(ax[0,0])
# cax = divider.append_axes("right", size="5%", pad=0.05)
# im = ax[0,0].imshow(asteriod_resize)
# fig.colorbar(im,cax ,ax=ax[0,0])
# ax[0,0].set_title('Object\nFlux [e-]: %f' %(np.sum(asteriod_resize)), fontsize = '16')

# divider = make_axes_locatable(ax[0,1])
# cax = divider.append_axes("right", size="5%", pad=0.05)
# im1 = ax[0,1].imshow(noise)
# fig.colorbar(im1,cax ,ax=ax[0,0])
# ax[0,1].set_title('Sum of the noise\nFlux [e-]: %f' %(np.sum(noise)), fontsize = '16')

# divider = make_axes_locatable(ax[1,0])
# cax = divider.append_axes("right", size="5%", pad=0.05)
# im2 = ax[1,0].imshow(img)
# fig.colorbar(im2,cax ,ax=ax[1,0])
# ax[1,0].set_title('Observed image\nFlux[e-]: %f' %(np.sum(img)), fontsize = '16')

# divider = make_axes_locatable(ax[1,1])
# cax = divider.append_axes("right", size="5%", pad=0.05)
# im3 = ax[1,1].imshow(img-np.fft.fftshift(conv_obj)-noise)
# fig.colorbar(im3,cax ,ax=ax[1,0])
# ax[1,1].set_title('Residual\nFlux[e-] Difference: %f %%' %(100*np.sum(img-conv_obj-noise)/np.sum(asteriod_resize)), fontsize = '16')


In [ ]:
# fig, ax = plt.subplots(2,2)

# rcParams['figure.figsize'] = 33 ,24
# divider = make_axes_locatable(ax[0,0])
# cax = divider.append_axes("right", size="5%", pad=0.05)
# im = ax[0,0].imshow(asteriod_resize)
# fig.colorbar(im,cax ,ax=ax[0,0])
# ax[0,0].set_title('Object\nFlux [e-]: %f' %(np.sum(asteriod_resize)), fontsize = '16')

# divider = make_axes_locatable(ax[0,1])
# cax = divider.append_axes("right", size="5%", pad=0.05)
# im1 = ax[0,1].imshow(noise)
# fig.colorbar(im1,cax ,ax=ax[0,0])
# ax[0,1].set_title('Sum of the noise\nFlux [e-]: %f' %(np.sum(noise)), fontsize = '16')

# divider = make_axes_locatable(ax[1,0])
# cax = divider.append_axes("right", size="5%", pad=0.05)
# im2 = ax[1,0].imshow(img)
# fig.colorbar(im2,cax ,ax=ax[1,0])
# ax[1,0].set_title('Observed image\nFlux[e-]: %f' %(np.sum(img)), fontsize = '16')

# divider = make_axes_locatable(ax[1,1])
# cax = divider.append_axes("right", size="5%", pad=0.05)
# im3 = ax[1,1].imshow(img-np.fft.fftshift(conv_obj)-noise)
# fig.colorbar(im3,cax ,ax=ax[1,0])
# ax[1,1].set_title('Residual\nFlux[e-] Difference: %f %%' %(100*np.sum(img-conv_obj-noise)/np.sum(asteriod_resize)), fontsize = '16')


In [ ]:
# What variables to be minimised
param_mask = np.asarray([1,0,1,0,0])
hyper_param_mask = np.asarray([1,1,0])

mask = np.concatenate((param_mask,hyper_param_mask))

from deconvbench.stat import DSPFit, Circmoyto2D
from deconvbench import RegulPSD

# rho, psd_param, _, psd1d = DSPFit(obs_image)

# # psd_param = K, rho0, p

# plt.imshow(obs_image)
# psd1 = Circmoyto2D(rho,psd1d,obs_image.shape[0])

# hyper_guess = psd_param
# print("PSD param: ",hyper_guess)

# dspb = np.sum(np.abs(np.fft.fft2(noise))**2)
# mu = dspb/psd_param[0]

# print("What is mu?", mu)

hyper_guess = amiralparam.hyperparam_initial(psf_guess)
# amiralparam.hyperparam_initial(psf_guess)
# print(hyper_guess)
hyper_min, hyper_max = amiralparam.hyperparam_bound(psf_guess, p_upperbound = 100.)
print(hyper_min,hyper_max)

psf_guess[-3] = hyper_guess[0]
psf_guess[-2] = hyper_guess[1] 
psf_guess[-1] = hyper_guess[2] 


# psf_guess[-3] = mu
# psf_guess[-1] = 2.91
# psf_guess[-2] = 1.32*()


# psf_guess[-3] = 2.01197235e-08 
# psf_guess[-2] = 0.82*(DIMENSION)/512
# psf_guess[-1] = 2.91

param_min = np.asarray([0.05,0,0,1e-8,1.01])
param_max =  np.asarray([1.,1e8,1e8,1e3,10])

upperbound = np.concatenate((param_max, hyper_max))
lowerbound = np.concatenate((param_min, hyper_min))

param_numerical_condition = np.array([1., 1e-4, 1., 1., 1.])
hyperparam_numerical_condition = np.array([hyper_guess[0], hyper_guess[1], 1.])

numerical_condition = np.concatenate((param_numerical_condition, hyperparam_numerical_condition))

amiral_cls = parameter.amiral(img=img, guess=psf_guess, aosys = aosys_cls, upperbound = upperbound, lowerbound= lowerbound, numerical_condition = numerical_condition, fourier_variable = amiralparam.fourier_variable, mask = mask)

In [ ]:
# amiralparam.fourier_variable

In [ ]:
np.shape(amiralparam.fourier_variable['ft(psf)'])

In [ ]:
psf_guess

In [ ]:
est_criterion, value_criterion, value_grad = amiral_cls.minimisation(psf_guess)

In [ ]:
print(est_criterion)

In [ ]:
est_otf_atmo, est_otf = create_psfao19_otf(otf_tel, est_criterion[0:5], aosys_cls)
# est_psf = np.fft.fftshift(binning(np.fft.fftshift(np.real(np.fft.ifft2(np.fft.ifftshift(est_otf)))), aosys_cls.samp_factor[0]))
est_psf = np.fft.fftshift(np.fft.fftshift(np.real(np.fft.ifft2(np.fft.ifftshift(est_otf)))))

plt.imshow(np.log10(psf_total))

In [ ]:
binned_psf_tel = binning(np.real(psf_tel), aosys_cls.samp_factor[0])

est_SR = np.max(np.real(est_psf)) / np.max(np.real(psf_tel))

print(np.sum(est_psf), np.sum(binned_psf_tel))

SR = np.max(psf_binned) / np.max(binned_psf_tel)

print(SR)

In [ ]:
diff_SR = np.real(est_SR-SR)*100
print(diff_SR)

In [ ]:
est_SR

In [ ]:
np.max(np.real(est_psf)) / np.max(np.real(binned_psf_tel))

In [ ]:
np.sum(est_psf) - np.sum(psf_binned)

In [ ]:
# est_psf = np.real(np.fft.ifft2(np.fft.ifftshift(est_otf_total)))

Intake the estimated criterion for plotting the PSF

In [ ]:
est_param = est_criterion[0:5]
est_otf_atmo, est_otf_total = create_psfao19_otf(otf_tel,est_param,aosys_cls)

from amiral.plotting import plotting_PSF_PSD
# Plot the PSF slice
rcParams['figure.figsize'] = 11,9

ycent = int((est_psf.shape[0]//2))
    
fig, ax = plt.subplots()
ax.set_title(r"PSF Estimation (with noise)$\mathrm{ \{r_0, sig^2, mu, \rho_0 \}}$"
             "\nSR Error: %.2f %%\nFlux[e-]: %.2E" %(diff_SR,flux_snr),fontsize = 18)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylabel(r'$\mathrm{Mean Intensity [e^-]}$', fontsize = 18)
ax.set_xlabel(r'$\mathrm{Position [pixel]}$', fontsize = 18)
# ax.plot(utils.mean_cir_array(np.real(psf_total)), label = "True")
# ax.plot(utils.mean_cir_array(np.real(est_psf)), label = "Estimated PSF")
ax.plot(((psf_total)[ycent, ycent:]), label = "True")
ax.plot(((est_psf)[ycent, ycent:]),label = "Estimated PSF")

# ax.plot(utils.mean_cir_array(np.real(psf_total)), label = "True")
# ax.plot(utils.mean_cir_array(np.real(est_psf)), label = "Estimated PSF")

ax.tick_params(axis='both', which='major', labelsize=18)
ax.legend()

# fig.savefig('/Users/alau/Data/amiral_fits/VESTA/SNR/2021apr06/deconv/VESTA_PSF_Estimation.pdf')


In [ ]:
est_param = est_criterion[0:5]
est_otf_atmo, est_otf_total = create_psfao19_otf(otf_tel,est_param,aosys_cls)

from amiral.plotting import plotting_PSF_PSD
# Plot the PSF slice
rcParams['figure.figsize'] = 11,9

ycent = int((psf_total.shape[0]//2))
    
fig, ax = plt.subplots()
ax.set_title(r"OTF Estimation (with noise)$\mathrm{ \{r_0, sig^2, mu, \rho_0 \}}$"
             "\nSR Error: %.2f %%\nFlux[e-]: %.2E" %(diff_SR,flux_snr),fontsize = 18)
ax.set_xscale('log')
# ax.set_yscale('log')
# ax.set_xlim(0)
ax.set_ylabel(r'$\mathrm{Intensity}$', fontsize = 18)
ax.set_xlabel(r'$\mathrm{Frequency}$', fontsize = 18)



ax.plot((np.abs(np.fft.fftshift(otf_total))[ycent, ycent:]), label = "True")
ax.plot(np.abs(est_otf_total)[ycent, ycent:],label = "Estimated")


# ax.plot(utils.circavg(np.abs(np.fft.fftshift(otf_total))))
# ax.plot(utils.circavg(np.abs(est_otf_total)), label = "Estimated")
ax.tick_params(axis='both', which='major', labelsize=18)
ax.legend()

# fig.savefig('/Users/alau/Data/amiral_fits/VESTA/SNR/2021apr06/deconv/VESTA_PSF_Estimation.pdf')



In [ ]:
plt.plot(np.abs(np.fft.fftshift(otf_total))[ycent, ycent:])
plt.plot(np.abs(est_otf_total)[ycent, ycent:])


print(np.max(np.abs(est_otf_total)))

In [ ]:
plt.imshow(np.real(np.log10(np.fft.fftshift(est_otf_total))))

In [ ]:
import time

start_time = time.time()
dec = Deconvbench(obs_image,est_psf,ron = 15)
dec.verbose_modulo = 10 # print every 10 iteration
dec.regularization.scale *= 4. # sharpen details (reduce regularization)
estim = dec.run()
end_time = time.time()
runtime = end_time - start_time

print("Run Time (mintues): ",runtime/60)

# hdr = write2header (est_criterion,0.,0.,amiral_keys) 

# fits.writeto("test_vesta"+'.fits', estim, hdr)

In [ ]:



norm_estim = estim / np.sum(obs_image)

ft_estim = np.fft.fftshift(np.fft.fft2(estim))
ft_estim = ft_estim/np.max(ft_estim)



norm_asteriod_resize = asteriod_resize / np.sum(obs_image)

ft_object = np.fft.fftshift(np.fft.fft2(asteriod_resize))
ft_object = ft_object/np.max(ft_object)

norm_obs_image = obs_image / np.sum(obs_image)

ft_obs_image = np.fft.fftshift(np.fft.fft2(obs_image))
ft_obs_image = ft_obs_image/np.max(ft_obs_image)

ft_est_psf = np.fft.fftshift(np.fft.fft2(est_psf))
print("Sum:",np.sum(est_psf))
ft_psf_total = np.fft.fftshift(np.fft.fft2(psf_total))
print("Sum:",np.sum(psf_total))


sum_img_flux = np.sum(obs_image)
sum_est_flux = np.sum(estim)

print("Diff %% ", 100*(np.sum(obs_image-estim))/np.sum(asteriod_resize))

In [ ]:
plt.imshow(obs_image-estim, cmap = 'gray')

Right, we have the deconvolved object now. Is the flux conserved? 

In [ ]:
print("Flux of the image [e-]: %f" %(np.sum(obs_image)))
print("Flux of the deconvolved object: %f" %(np.sum(estim)))
print("Flux difference [%%]: %f" %(np.sum(obs_image-estim)/np.sum(obs_image)))

In [ ]:
fXY = aosys_cls.fX**2 +  aosys_cls.fY**2 
fxy = _fxy[0]**2 + _fxy[1]**2

plt.imshow(fXY-fxy)

In [ ]:
plt.plot(utils.mean_cir_array((np.abs(ft_obs_image)**2)))

In [ ]:
plt.imshow(np.log10((np.abs(ft_estim))-(np.abs(ft_object))))

print(((np.max(np.abs(ft_estim))-np.max(np.abs(ft_object)))/(np.max(np.abs(ft_object)))))

In [ ]:
est_param = est_criterion[0:5]
est_otf_atmo, est_otf_total = create_psfao19_otf(otf_tel,est_param,aosys_cls)

print(utils.mean_cir_array((np.abs(ft_object)))[0],utils.mean_cir_array((np.abs(ft_obs_image)))[0],utils.mean_cir_array((np.abs(ft_estim)))[0])
from amiral.plotting import plotting_PSF_PSD

# Plot the PSF slice
rcParams['figure.figsize'] = 11,9

ycent = int((psf_total.shape[0]//2))
    
fig, ax = plt.subplots()
ax.set_title(r"|FFT(Object)|"
             "\nSR Error: %.2f %%\nFlux[e-]: %.2E" %(diff_SR,flux_snr),fontsize = 18)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylabel(r'$\mathrm{Norm_Intensity}$', fontsize = 18)
ax.set_xlabel(r'$\mathrm{Frequency}$', fontsize = 18)
# ax.set_xlim(np.min(utils.mean_cir_array(fXY)),np.max(utils.mean_cir_array(fXY)))

# print(fXY[ycent, ycent:]/np.max(fXY[ycent, ycent:]))


# ax.plot(fXY[ycent, ycent:],(np.abs(ft_estim))[ycent, ycent:], label = "Deconvolved")


# ax.plot(fXY[ycent, ycent:],(np.abs(ft_object))[ycent, ycent:], label = "True")

# ax.plot(utils.mean_cir_array(fXY),utils.mean_cir_array(np.abs(ft_estim)), label = "Estimated")
# ax.set_xlim(110, 120)

# ax.plot(utils.mean_cir_array(np.abs(np.fft.fftshift(np.fft.fft2(est_psf)))), label = "Estimated")
# ax.plot(utils.mean_cir_array(np.abs(np.fft.fftshift(np.fft.fft2(psf_total)))), label = "True")


ax.plot(utils.mean_cir_array(fXY),utils.mean_cir_array((np.abs(ft_obs_image))), label = "Observation")
ax.plot(utils.mean_cir_array(fXY),utils.mean_cir_array((np.abs(ft_estim))), label = "Estimated")
ax.plot(utils.mean_cir_array(fXY),utils.mean_cir_array((np.abs(ft_object))), label = "Object")




ax.tick_params(axis='both', which='major', labelsize=18)
ax.legend()

In [ ]:
# zoom into the array

# zoom_residual = zoom_array(estim-asteriod_resize, 200)

fig, ax = plt.subplots()
rcParams['figure.figsize'] = 10 ,10
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
im = ax.imshow((estim-asteriod_resize), cmap = 'gray')
cbar = fig.colorbar(im, cax=cax,ax=ax)
cbar.ax.tick_params(labelsize=14)
ax.set_title('Residual (Estimated_PSF - True Object)\nDiff(SR) = %.2f %%\nFlux Difference: %.2f %%\nr0 = %.2f cm sig2 = %.2f'
             '\nFlux[e-]: %.2E'%(diff_SR,np.sum(obs_image-estim)/np.sum(obs_image), 100*psf_param[0], psf_param[2], flux_snr), fontsize = 18)
ax.tick_params(axis='both', which='major', labelsize=14)

# # fig.savefig('/Users/alau/Data/amiral_fits/VESTA/SNR/2021apr06/deconv/VESTA_noise_free_residual.pdf')

In [ ]:
plt.imshow(zoom_array(estim, 200))

In [ ]:
import time

start_time = time.time()

dec = Deconvbench(obs_image,np.fft.fftshift(psf_binned),ron = 15)
dec.verbose_modulo = 100 # print every 10 iteration
dec.regularization.scale *= 2. # sharpen details (reduce regularization)
estim_1 = dec.run()

end_time = time.time()
runtime = end_time - start_time

print("Run Time (mintues): ",runtime/60)

In [ ]:
perfect_deconv = estim_1

zoom_diff = zoom_array(estim-perfect_deconv, 200)
zoom_deconv = zoom_array(perfect_deconv, 200)

fig, ax = plt.subplots(1,2)

rcParams['figure.figsize'] = 16,19

divider = make_axes_locatable(ax[0])
cax = divider.append_axes("right", size="5%", pad=0.05)
im = ax[0].imshow(perfect_deconv)
fig.colorbar(im,cax ,ax=ax[0])
ax[0].set_title('L2-L1 Deconvolved with the exact PSF''\nFlux [e-]: %.2E' %(np.sum(perfect_deconv)), fontsize = 14)

divider = make_axes_locatable(ax[1])
cax = divider.append_axes("right", size="5%", pad=0.05)
im1 = ax[1].imshow(perfect_deconv-estim)
fig.colorbar(im1, cax ,ax=ax[1])
ax[1].set_title('Residual (Estimated_PSF - True_PSF)\nDiff(SR) = %.2f %%\nFlux Difference: %.2f %%\nr0 = %.2f cm sig2 = %.2f'
             %(diff_SR,np.sum(obs_image-perfect_deconv)/np.sum(obs_image), 100*psf_param[0], psf_param[2]), fontsize = 14)


print(100*np.sum(obs_image-estim)/np.sum(obs_image))


In [ ]:
# fig, ax = plt.subplots()

# rcParams['figure.figsize'] = 33 ,24

# im = ax.imshow(np.abs(estim-perfect_deconv))
# fig.colorbar(im, ax=ax)
# ax.set_title('Exact PSF Residual(Normalised)\nDiff(SR) = 2.16%')

In [ ]:
# fig, ax = plt.subplots()

# rcParams['figure.figsize'] = 33 ,24

# im = ax.imshow(asteriod_resize)
# fig.colorbar(im, ax=ax)
# ax.set_title('Object')

In [ ]:
from deconvbench.stat import DSPFit, Circmoyto2D
from deconvbench import RegulPSD

rho, psd_param, _, psd1d = DSPFit(obs_image)
psd1 = Circmoyto2D(rho,psd1d,obs_image.shape[0])

hyper = est_crtierion[-3:-1]


psd_object = psd_object(hyper)




# plot PSD

rcParams['figure.figsize'] = 13 ,11

ycent = int((psf_total.shape[0]//2))
    
fig, ax = plt.subplots()
ax.set_xscale('log')
ax.set_yscale('log')
ax.plot(utils.mean_cir_array(np.real(psd1)), label = "True")
# ax.plot(utils.mean_cir_array(np.real(est_psf)), label = "Estimated PSF")

ax.legend()

print(psd_param)




In [ ]:
# #%% ITERATIVE DECONVOLUTION
# dec_psd1 = Deconvbench(obs_image, est_psf, ron=10, positivity=False, verbose=True)
# dec_psd1.verbose_modulo = 50
# dec_psd1.regularization = RegulPSD(psd1) # set PSD regularization
# estD1 = dec_psd1.run()

In [ ]:
# fig, ax = plt.subplots()

# rcParams['figure.figsize'] = 33 ,24

# im = ax.imshow(estD1)
# fig.colorbar(im, ax=ax)
# ax.set_title('Deconv')

In [ ]:
# from deconvbench.stat import DSPFit, Circmoyto2D
# from deconvbench import RegulPSD

# rho, psd_param, _, psd1d = DSPFit(obs_image)
# psd1 = Circmoyto2D(rho,psd1d,obs_image.shape[0])


# #%% ITERATIVE DECONVOLUTION
# dec_psd = Deconvbench(obs_image, psf_total, ron=10, positivity=False, verbose=True)
# dec_psd.verbose_modulo = 50
# dec_psd.regularization = RegulPSD(psd1) # set PSD regularization
# estD = dec_psd.run()

In [ ]:
# fig, ax = plt.subplots(1,3)

# rcParams['figure.figsize'] = 33 ,24

# divider = make_axes_locatable(ax[0])
# cax = divider.append_axes("right", size="5%", pad=0.05)
# im = ax[0].imshow(estD)
# fig.colorbar(im, cax, ax = ax[0])
# ax[0].set_title('PSD Regularisation (With exact PSF)',fontsize = '18')

# divider = make_axes_locatable(ax[1])
# cax = divider.append_axes("right", size="5%", pad=0.05)
# im1 = ax[1].imshow(estD1)
# fig.colorbar(im1, cax, ax = ax[1])
# ax[1].set_title('PSD Regularisation (With estimated PSF)',fontsize = '18')


# divider = make_axes_locatable(ax[2])
# cax = divider.append_axes("right", size="5%", pad=0.05)
# im2 = ax[2].imshow(obs_image)
# fig.colorbar(im2, cax ,ax=ax[2])
# ax[2].set_title('Observed Image',fontsize = '18')


